In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import re

In [16]:
consDF = pd.read_parquet('/uss/hdsi-prismdata/q2-ucsd-consDF.pqt')
consDF

,prism_consumer_id,evaluation_date,credit_score,DQ_TARGET
0,0,2021-09-01,726.0,0.0
1,1,2021-07-01,626.0,0.0
2,2,2021-05-01,680.0,0.0
3,3,2021-03-01,734.0,0.0
4,4,2021-10-01,676.0,0.0
...,...,...,...,...
14995,14995,2022-03-08,655.0,NaN
14996,14996,2022-01-15,625.0,NaN
14997,14997,2022-01-31,688.0,NaN
14998,14998,2022-03-08,722.0,NaN


In [6]:
acctDF = pd.read_parquet('/uss/hdsi-prismdata/q2-ucsd-acctDF.pqt')
acctDF 

,prism_consumer_id,prism_account_id,account_type,balance_date,balance
0,3023,0,SAVINGS,2021-08-31,90.57
1,3023,1,CHECKING,2021-08-31,225.95
2,4416,2,SAVINGS,2022-03-31,15157.17
3,4416,3,CHECKING,2022-03-31,66.42
4,4227,4,CHECKING,2021-07-31,7042.90
...,...,...,...,...,...
24461,11500,24461,CHECKING,2022-03-27,732.75
24462,11615,24462,SAVINGS,2022-03-30,5.00
24463,11615,24463,CHECKING,2022-03-30,1956.46
24464,12210,24464,CHECKING,2022-03-28,2701.51


In [7]:
len(acctDF['prism_consumer_id'].unique())

13009

In [8]:
trxnDF= pd.read_parquet('/uss/hdsi-prismdata/q2-ucsd-trxnDF.pqt')
trxnDF



,prism_consumer_id,prism_transaction_id,category,amount,credit_or_debit,posted_date
0,3023,0,4,0.05,CREDIT,2021-04-16
1,3023,1,12,481.56,CREDIT,2021-04-30
2,3023,2,4,0.05,CREDIT,2021-05-16
3,3023,3,4,0.07,CREDIT,2021-06-16
4,3023,4,4,0.06,CREDIT,2021-07-16
...,...,...,...,...,...,...
6407316,10533,6405304,31,4.96,DEBIT,2022-03-11
6407317,10533,6405305,12,63.48,DEBIT,2022-03-30
6407318,10533,6405306,12,53.99,DEBIT,2022-03-30
6407319,10533,6405307,12,175.98,DEBIT,2022-03-31


In [9]:
acctDF = pd.read_parquet('/uss/hdsi-prismdata/q2-ucsd-acctDF.pqt')
acctDF 

,prism_consumer_id,prism_account_id,account_type,balance_date,balance
0,3023,0,SAVINGS,2021-08-31,90.57
1,3023,1,CHECKING,2021-08-31,225.95
2,4416,2,SAVINGS,2022-03-31,15157.17
3,4416,3,CHECKING,2022-03-31,66.42
4,4227,4,CHECKING,2021-07-31,7042.90
...,...,...,...,...,...
24461,11500,24461,CHECKING,2022-03-27,732.75
24462,11615,24462,SAVINGS,2022-03-30,5.00
24463,11615,24463,CHECKING,2022-03-30,1956.46
24464,12210,24464,CHECKING,2022-03-28,2701.51


In [10]:
acctDF

,prism_consumer_id,prism_account_id,account_type,balance_date,balance
0,3023,0,SAVINGS,2021-08-31,90.57
1,3023,1,CHECKING,2021-08-31,225.95
2,4416,2,SAVINGS,2022-03-31,15157.17
3,4416,3,CHECKING,2022-03-31,66.42
4,4227,4,CHECKING,2021-07-31,7042.90
...,...,...,...,...,...
24461,11500,24461,CHECKING,2022-03-27,732.75
24462,11615,24462,SAVINGS,2022-03-30,5.00
24463,11615,24463,CHECKING,2022-03-30,1956.46
24464,12210,24464,CHECKING,2022-03-28,2701.51


In [33]:
len(acctDF['prism_consumer_id'].unique())


13009

In [34]:
len(acctDF['prism_account_id'].unique())

24466

In [11]:
acct_features = acctDF.groupby(
    ['prism_consumer_id', 'prism_account_id']
).agg(
    min_balance=('balance', 'min'),
    median_balance=('balance', 'median'),
    balance_std=('balance', 'std')
).reset_index()

In [14]:
consumer_balance_features = acct_features.groupby('prism_consumer_id').agg(
    min_balance=('min_balance', 'min'),          # worst-case buffer
    median_balance=('median_balance', 'median'), # typical buffer
    balance_volatility=('balance_std', 'median') # instability
).reset_index()

In [21]:
model_df = consumer_balance_features.merge(
    consDF[['prism_consumer_id', 'DQ_TARGET']],
    on='prism_consumer_id',
    how='inner'
)

model_df = model_df.dropna(subset=['DQ_TARGET'])

In [22]:
features = {
    'min_balance': ['min_balance'],
    'median_balance': ['median_balance'],
    'balance_volatility': ['balance_volatility']
}

In [45]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_score
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import roc_auc_score, recall_score, roc_curve, precision_score, f1_score


In [56]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score, average_precision_score, precision_score, 
    recall_score, f1_score
)

results = []

y = model_df['DQ_TARGET']

#
threshold = 0.1

for name, cols in features.items():
    X = model_df[cols].fillna(0)

    # Split data, maintain class balance
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )

    model = LogisticRegression()
    model.fit(X_train, y_train)

    # predict prob positive class
    y_probs = model.predict_proba(X_test)[:, 1]

    # Binarize predictions using threshold
    y_pred = (y_probs >= threshold).astype(int)

    
    results.append({
        'feature': name,
        'roc_auc': roc_auc_score(y_test, y_probs),
        'pr_auc': average_precision_score(y_test, y_probs),
        'precision': precision_score(y_test, y_pred, zero_division=0),
        'recall': recall_score(y_test, y_pred, zero_division=0),
        'f1': f1_score(y_test, y_pred, zero_division=0)
    })

results_df = pd.DataFrame(results).sort_values('roc_auc', ascending=False)
print(results_df)



              feature   roc_auc    pr_auc  precision    recall        f1
1      median_balance  0.716187  0.194185   0.131030  0.800000  0.225179
0         min_balance  0.669789  0.151326   0.129375  0.752727  0.220800
2  balance_volatility  0.500000  0.088056   0.000000  0.000000  0.000000


In [ ]:
#median_balance

# model prioritizes catching delinquents (high recall) but predicts too many 
#false positives (low precision)

#ROC-AUC median_balance helps rank consumers correctly
#High balances - less likely to default, low balances- more likely


#Precision-predicted positives (DQ=1), only 13% were actually delinquent

#Recall- got 80% of actual delinquents

#F1-moderate, limited by low precision

In [ ]:
# min balance 

#ROC-AUC - Still predictive, but less powerful than median_balance
#Min balance more extreme but might be a rare event, so it ranks consumers less accurately

#PR-AUC-Lower-fewer true positives relative to predicted positives

#Precision-Similar low precision as median_balance

#Recall- High recall captures most delinquents, but again with false positives

#F1-lower than median balance


In [ ]:
#balance_volatility-


#ROC-AUC - Still predictive, but less powerful than median_balance
#Min balance more extreme but might be a rare event, so it ranks consumers less accurately

#PR-AUC-Lower-very low, worse

#Precision/Recall/F1- all 0, cannot separate delinquents at all using volatility alone
#

In [ ]:
#median_balance -typical amount of money a consumer keeps in their accounts (checking + savings)
# median is robust to extreme deposits or withdrawals 
#Consumers with consistently low median balances often have less financial cushion, which makes them more likely to fall behind on payments (DQ_TARGET = 1).

In [ ]:
#Median balancep capturing the typical financial buffer a consumer maintains
